In [ ]:
import os
os.makedirs('my_rag_project', exist_ok=True)

In [ ]:

os.makedirs('my_rag_project/documents', exist_ok=True)

In [ ]:
os.chdir('/content/my_rag_project')

In [ ]:
!ls -la


total 16
drwxr-xr-x 4 root root 4096 Aug 20 02:31 .
drwxr-xr-x 1 root root 4096 Aug 20 02:06 ..
drwxr-xr-x 3 root root 4096 Aug 20 02:32 db
drwxr-xr-x 2 root root 4096 Aug 20 02:31 documents


In [ ]:
!pip install google-generativeai
!pip install langchain
!pip install langchain-google-genai
!pip install chromadb
!pip install PyMuPDF
!pip install python-dotenv
!pip install langchain-community
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.9 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
import google.generativeai as genai
import langchain
import chromadb
import fitz  # PyMuPDF
from dotenv import load_dotenv
print("All packages installed successfully!")

All packages installed successfully!


In [ ]:
# Access your secret
from google.colab import userdata
MY_GEMINI_KEY = userdata.get('MY_GEMINI_KEY')

In [ ]:
# Set up Gemini
import google.generativeai as genai
import os
print("API key configured successfully!")


os.environ["GOOGLE_API_KEY"] = MY_GEMINI_KEY
genai.configure(api_key=MY_GEMINI_KEY)

API key configured successfully!


In [ ]:
# Check what's in your documents directory
!ls -la documents/

total 2164
drwxr-xr-x 2 root root    4096 Aug 20 02:31 .
drwxr-xr-x 4 root root    4096 Aug 20 02:31 ..
-rw-r--r-- 1 root root 2206753 Aug 20 02:31 3-17_NERVOUS_HANDOUT.pdf


In [ ]:
from google.colab import files
import shutil

print("Upload your PDF files:")
uploaded = files.upload()

# Move uploaded files to the documents directory
for filename in uploaded.keys():
    if filename.endswith('.pdf'):
        shutil.move(filename, f'documents/{filename}')
        print(f"✅ Moved {filename} to documents/")
    else:
        print(f"⚠️ {filename} is not a PDF file, skipping...")

# Check what's now in documents
print("\nDocuments directory now contains:")
!ls -la documents/

Upload your PDF files:


Saving 3-17_NERVOUS_HANDOUT.pdf to 3-17_NERVOUS_HANDOUT.pdf
✅ Moved 3-17_NERVOUS_HANDOUT.pdf to documents/

Documents directory now contains:
total 2164
drwxr-xr-x 2 root root    4096 Aug 20 02:42 .
drwxr-xr-x 4 root root    4096 Aug 20 02:42 ..
-rw-r--r-- 1 root root 2206753 Aug 20 02:42 3-17_NERVOUS_HANDOUT.pdf


In [ ]:
import os
from langchain.vectorstores import Chroma
from langchain.document_loaders import AsyncChromiumLoader #directlyt import the data from an URL
from langchain.document_transformers import Html2TextTransformer #converts html to text data
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [ ]:
document = []
for filename in os.listdir("documents"):
    if filename.lower().endswith(".pdf"):
        loader = PyMuPDFLoader(os.path.join("documents", filename))
        document.extend(loader.load())

print(f"Total pages loaded: {len(document)}")

Total pages loaded: 12


In [ ]:
# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = text_splitter.split_documents(document)

len(text_chunks)

33

In [ ]:
# Use in embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=MY_GEMINI_KEY  # Use your variable name here
)

In [ ]:
# Create vector database
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=text_chunks,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
retriever = vectordb.as_retriever()


In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)


In [ ]:
### Function to Ask LLM First
def ask_llm_first(query):
    """Ask the LLM first. If it confidently answers, return it. Otherwise, proceed to VectorDB."""

    system_prompt = (
        "You are an AI assistant. Answer only if you are confident about your response. "
        "If you do not know the answer, simply respond with: 'I don't know.' "
        "Do not guess or make assumptions.\n\n"
        f"User Question: {query}"
    )

    response = llm.invoke(system_prompt)  # Query the LLM

    # Extract text content from AIMessage object
    if hasattr(response, "content"):
        response_text = response.content  # Correct way to get the response text
    else:
        response_text = str(response)  # Fallback in case 'content' is missing

    # Define phrases that indicate uncertainty
    vague_responses = [
        "i don't know", "i am not sure", "i need more information", "please provide more details",
        "i cannot determine", "i am unable to", "i do not have enough context",
        "i need more context", "i do not have enough information", "i require additional details"
    ]

     # Debugging: Print each phrase check
    for phrase in vague_responses:
        if phrase in response_text.lower():
            print(f"✅ Match found: '{phrase}' is in LLM response!")
            print("⚠️ LLM is unsure. Fetching data from VectorDB...")
            return False  # Indicate retrieval is needed

    print("🤖 LLM confidently answered without RAG.")
    return response_text  # Return LLM's answer

In [ ]:
### Function to Perform RAG Retrieval
def ask_with_rag(query):
    """Retrieve documents from VectorDB and generate an answer using LLM with context."""
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    response = qa_chain.invoke({"query": query})
    print("\n🤖 AI Agent's Response (with RAG):")
    return response["result"]


In [ ]:
def start_conversation():
    """Start an interactive chat session with the LLM and VectorDB."""
    print("\n💬 Start chatting with the AI! (Type 'exit' to stop)\n")

    while True:
        # Get user input dynamically
        user_query = input("👤 You: ").strip()

        # Exit condition
        if user_query.lower() == "exit":
            print("👋 Goodbye!")
            break

        # Step 1: Ask LLM first
        llm_response = ask_llm_first(user_query)

        # Step 2: If LLM is unsure (False), use VectorDB retrieval
        if llm_response is False:
            final_answer = ask_with_rag(user_query)
            print(f"\n🤖 AI: {final_answer}\n")
        else:
            print(f"\n🤖 AI: {llm_response}\n")

In [ ]:
start_conversation()



💬 Start chatting with the AI! (Type 'exit' to stop)

👤 You: what drugs affect the nervous system
🤖 LLM confidently answered without RAG.

🤖 AI: Many drugs affect the nervous system.  This is a very broad category.  To give a useful answer, I need more specifics.  What kind of effect are you interested in? (e.g., stimulants, depressants, analgesics, etc.)

👤 You: stimulants please
✅ Match found: 'i don't know' is in LLM response!
⚠️ LLM is unsure. Fetching data from VectorDB...

🤖 AI Agent's Response (with RAG):

🤖 AI: Based on the provided text, caffeine and nicotine are stimulants.  Caffeine stimulates the central nervous system by suppressing melatonin and promoting adrenalin. Nicotine also stimulates the central nervous system, causing an increased release of neurotransmitters associated with pleasure.

👤 You: what about anticonvulsants
✅ Match found: 'i don't know' is in LLM response!
⚠️ LLM is unsure. Fetching data from VectorDB...

🤖 AI Agent's Response (with RAG):

🤖 AI: I'm so

KeyboardInterrupt: Interrupted by user